In [ ]:
from datetime import datetime, date
import pandas as pd
from gs_quant.instrument import IRSwap
from gs_quant.common import BuySell, OptionType
from gs_quant.backtests.triggers import PeriodicTrigger, PeriodicTriggerRequirements
from gs_quant.backtests.actions import AddTradeAction, HedgeAction
from gs_quant.backtests.generic_engine import GenericEngine
from gs_quant.backtests.strategy import Strategy
from gs_quant.risk import Price, IRDelta

In [ ]:
# Initialize session
from gs_quant.session import GsSession
GsSession.use(client_id=None, client_secret=None, scopes=('run_analytics',)) 

## Simple backtest with no holiday calendar handling.

In [ ]:
# Define backtest dates
start_date = date(2024, 5, 3)
end_date = date(2024, 6, 3)

In [ ]:
# Define strategy  -- this strategy adds a swap every day holding forever.

trig_req = PeriodicTriggerRequirements(start_date=start_date, end_date=end_date, frequency='1b')
irswap = IRSwap('Pay', '10y', 'USD', notional_amount=10000, name='10y')
actions = AddTradeAction(irswap)
triggers = PeriodicTrigger(trig_req, actions)
strategy = Strategy(None, triggers)

In [ ]:
# run backtest daily
GE = GenericEngine()
backtest = GE.run_backtest(strategy, start=start_date, end=end_date, frequency='1b', show_progress=True)
backtest.result_summary

# Note that May 27th appears in result summary

In [ ]:
# we can see that a trade was opened on 27th May 2024 dispite it being a holiday because we didn't say anything about holidays in our strategy
date(2024, 5, 27) in list(backtest.trade_ledger()['Open'])

In [ ]:
# now change the trigger not to trigger on exclude May 27th as it was memorial day
holiday = (date(2024, 5, 27),)

trig_req_exclude_hol = PeriodicTriggerRequirements(start_date=start_date, end_date=end_date, frequency='1b', calendar=holiday)
triggers_exc_hol = PeriodicTrigger(trig_req_exclude_hol, actions)
strategy_exc_hol = Strategy(None, triggers_exc_hol)

backtest2 = GE.run_backtest(strategy_exc_hol, start=start_date, end=end_date, frequency='1b', show_progress=True)
backtest2.result_summary

# note May 27th still appears on result summary as valuing trades is done on the schedule created by run_backtest which didn't include this holiday

In [ ]:
# we can see that no trade was opened on 27th May 2024 because the trigger knew it wasn't a valid day
date(2024, 5, 27) in list(backtest2.trade_ledger()['Open'])

In [ ]:
# now lets run the strategy above excluding holidays at the run_backtest level
backtest3 = GE.run_backtest(strategy_exc_hol, start=start_date, end=end_date, frequency='1b', show_progress=True, holiday_calendar=holiday)
backtest3.result_summary

# now we can see that May 27th is excluded from the result_summary.  No trades were created on this date and no risks run.

In [ ]:
# Hang on.  What if the action specifies an exit date for a trade and that falls on a holiday!?

actions_roll_daily = AddTradeAction(irswap, '1b')
triggers_roll_daily = PeriodicTrigger(trig_req_exclude_hol, actions_roll_daily)
strategy_roll_daily = Strategy(None, triggers_roll_daily)

backtest4 = GE.run_backtest(strategy_roll_daily, start=start_date, end=end_date, frequency='1b', show_progress=True, holiday_calendar=holiday)
backtest4.result_summary

# now we see that May 27th is back...  That is because the action didn't know that it couldn't exit a trade on that date.

In [ ]:
date(2024, 5, 27) in list(backtest4.trade_ledger()['Open'])

In [ ]:
date(2024, 5, 27) in list(backtest4.trade_ledger()['Close'])

In [ ]:
# so now lets tell the action that 27th May is a holiday

actions_roll_daily_hol = AddTradeAction(irswap, '1b', holiday_calendar=holiday)
triggers_roll_daily_hol = PeriodicTrigger(trig_req_exclude_hol, actions_roll_daily_hol)
strategy_roll_daily_hol = Strategy(None, triggers_roll_daily_hol)

backtest5 = GE.run_backtest(strategy_roll_daily_hol, start=start_date, end=end_date, frequency='1b', show_progress=True, holiday_calendar=holiday)
backtest5.result_summary


# success